In [46]:
from io import BytesIO
import requests
import pandas as pd
import linearmodels

import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.stats as stats
from scipy.stats import norm
from matplotlib.dates import DateFormatter
import statsmodels.api as sm
import math as mth
from scipy.stats import pearsonr
from scipy.stats import spearmanr


from scipy.stats import f_oneway
from sklearn.feature_selection import f_regression
from linearmodels.panel import PanelOLS
from statsmodels.stats.diagnostic import het_goldfeldquandt
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.diagnostic import het_white

import plotly.graph_objects as go
import scipy.stats as stats
import datetime as dt

from scipy import stats as st

import warnings
warnings.filterwarnings("ignore")

font_1 = {'family': 'sans-serif',
        'color':  '#000000',
        'weight': 'normal',
        'size': 20,
        }

font_2 = {'family': 'sans-serif',
        'color':  '#000000',
        'weight': 'normal',
        'size': 15,
          
        }

---

# Data preprocessing

In [47]:
spreadsheet_id = '1khUr-TV24VXP8kQoRfCk7D9_1mJhMuWyzhwJ_EWT1xc'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
population_growth = pd.read_csv(BytesIO(r.content))
# population_growth.info()

In [48]:
spreadsheet_id = '1z9YsYGM4UztrBo9i5kDb_jgfM-3ctnO8FRn4hWE2bc8'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
migration = pd.read_csv(BytesIO(r.content))
# migration.info()

In [49]:
spreadsheet_id = '1-cHwRDRgd-70YmRs-RwImiaebbze8TDu_fVOuWwbgQo'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
cpi = pd.read_csv(BytesIO(r.content))
# cpi.info()

In [50]:
spreadsheet_id = '1lYwWgJd4DtVCSThBz7wiH5xYt-p2arn1Z-r2iZvRb34'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
gni_and_gdp = pd.read_csv(BytesIO(r.content))
# gni_and_gdp.info()

In [51]:
spreadsheet_id = '1yfqg_YUA5qeU5f8g1aK31YvfjR3Z4dBAoUBYxUmirEI'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
hdi = pd.read_csv(BytesIO(r.content))
# hdi.info()

In [52]:
spreadsheet_id = '1WYjtSbTfzBYdYbaOpVtpg0-e_IEhC6HvAeXnAbkLGto'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
labour_force = pd.read_csv(BytesIO(r.content))
# labour_force

In [53]:
spreadsheet_id = '1JIhGWsPWnFObd1ugfqvnYD5wTjk7DSxo0kgV1pZqKmA'
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)
wages = pd.read_csv(BytesIO(r.content))
# wages

---

### Merging all tables

In [54]:
migration_long = pd.melt(migration, id_vars=["year"], var_name="Region", value_name="migration")
population_growth_long = pd.melt(population_growth, id_vars=["year"], var_name="Region", value_name="population_growth")
cpi_long = pd.melt(cpi, id_vars=["year"], var_name="Region", value_name="cpi")
gni_and_gdp_long = pd.melt(gni_and_gdp, id_vars=["year"], var_name="Region", value_name="gni_and_gdp")
hdi_long = pd.melt(hdi, id_vars=["year"], var_name="Region", value_name="hdi")
labour_force_long = pd.melt(labour_force, id_vars=["year"], var_name="Region", value_name="labour_force")
wages_long = pd.melt(wages, id_vars=["year"], var_name="Region", value_name="wages")

In [55]:
# Merge all of the long-format tables into a single DataFrame
merged = migration_long.merge(
    population_growth_long, on=["year", "Region"]).merge(
    cpi_long, on=["year", "Region"]).merge(
    gni_and_gdp_long, on=["year", "Region"]).merge(
    hdi_long, on=["year", "Region"]).merge(
    labour_force_long, on=["year", "Region"]).merge(
wages_long, on=['year', "Region"])

merged.head()

,year,Region,migration,population_growth,cpi,gni_and_gdp,hdi,labour_force,wages
0,2010,akm,-5581.0,3728.0,100.083651,1406718.30,0.798,86.0,33221.0
1,2011,akm,-5706.0,3824.0,100.105166,1412013.59,0.806,85.7,39864.0
2,2012,akm,-3429.0,4799.0,100.138169,1419619.73,0.814,86.9,45002.0
3,2013,akm,-1975.0,4819.0,100.172401,1422898.11,0.823,87.2,47438.0
4,2014,akm,-4352.0,5300.0,100.183447,1422961.74,0.829,88.8,52771.0


---

# Analysis

1. Explore the data: Use Pandas functions like head(), describe(), and info() to get a basic understanding of the structure, distribution, and missing values of your data. Also using visualization tools like Plotly to create plots and histograms to visualize the distributions of your variables.


2. Correlation analysis: Calculate the correlation coefficients between migration and each of the other economic variables using the corr() function in Pandas. This gives an idea of the strength and direction of the relationships between migration and the other variables. Also create scatterplots to visualize these relationships.


3. Regression analysis: Use regression analysis to model the relationship between migration and the other economic variables, while controlling for other factors that may influence migration (such as regional differences, time trends, or other economic or social factors). Using tools like the statsmodels package in Python to perform multiple regression analysis and examine the significance and magnitude of the effects of each independent variable.


4. Interpret the results: After performing regression analysis, we move to interpreting the results to draw conclusions about the relationships between migration and the other economic variables. Also using the regression model to make predictions about how migration may change in response to changes in the economic variables.

---

## 1. Explore the data

### General info about our final table: 

In [56]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168 entries, 0 to 167
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               168 non-null    int64  
 1   Region             168 non-null    object 
 2   migration          168 non-null    float64
 3   population_growth  168 non-null    float64
 4   cpi                168 non-null    float64
 5   gni_and_gdp        168 non-null    float64
 6   hdi                168 non-null    float64
 7   labour_force       168 non-null    float64
 8   wages              168 non-null    float64
dtypes: float64(7), int64(1), object(1)
memory usage: 13.1+ KB


In [57]:
merged.describe()

,year,migration,population_growth,cpi,gni_and_gdp,hdi,labour_force,wages
count,168.000000,168.000000,168.000000,168.000000,1.680000e+02,168.000000,168.000000,168.000000
mean,2015.500000,-2120.970238,11957.505952,100.261630,2.744583e+06,0.800310,83.126190,80282.541667
std,3.462373,9756.844056,8904.824314,0.273172,1.444689e+06,0.022471,3.701889,38510.023028
min,2010.000000,-20135.000000,-1932.000000,99.548564,1.033999e+06,0.747000,71.900000,27474.000000
25%,2012.750000,-6400.250000,5142.500000,100.089909,1.459856e+06,0.789000,80.500000,50808.000000
50%,2015.500000,-3228.000000,11073.500000,100.259797,2.436203e+06,0.798000,83.250000,71030.500000
75%,2018.250000,-581.750000,15935.250000,100.441382,3.105414e+06,0.812000,86.200000,103191.750000
max,2021.000000,40742.000000,38218.000000,100.906987,6.031893e+06,0.849000,91.200000,251597.000000


### Example of visualisation for migration variable:

In [82]:
dat = merged[['year', 'Region', 'migration']]
print(dat)

     year     Region  migration
0    2010        akm    -5581.0
1    2011        akm    -5706.0
2    2012        akm    -3429.0
3    2013        akm    -1975.0
4    2014        akm    -4352.0
..    ...        ...        ...
163  2017  north_kaz    -5463.0
164  2018  north_kaz    -4697.0
165  2019  north_kaz    -5943.0
166  2020  north_kaz    -4387.0
167  2021  north_kaz    -4755.0

[168 rows x 3 columns]


In [58]:
fig = go.Figure()

for region in merged['Region'].unique():
    data = merged[merged['Region'] == region]
    fig.add_trace(go.Scatter(x=data['year'], y=data['migration'], mode='lines', name=region))

fig.update_layout(title='Migration by Region and Year',
                  xaxis_title='Year',
                  yaxis_title='Migration')

fig.show()

## 2. Correlation analysis

In [59]:
corr_matrix = merged.corr()
display(corr_matrix)

,year,migration,population_growth,cpi,gni_and_gdp,hdi,labour_force,wages
year,1.000000,-0.102507,0.047734,0.641040,0.012032,0.610947,0.217800,0.654627
migration,-0.102507,1.000000,0.188372,-0.452017,-0.106719,0.120131,-0.313182,0.294946
population_growth,0.047734,0.188372,1.000000,-0.158754,-0.274862,-0.053735,-0.296764,0.047133
cpi,0.641040,-0.452017,-0.158754,1.000000,0.014456,0.240638,0.353020,0.283075
gni_and_gdp,0.012032,-0.106719,-0.274862,0.014456,1.000000,-0.014216,0.300251,0.199158
hdi,0.610947,0.120131,-0.053735,0.240638,-0.014216,1.000000,0.224949,0.460680
labour_force,0.217800,-0.313182,-0.296764,0.353020,0.300251,0.224949,1.000000,0.015378
wages,0.654627,0.294946,0.047133,0.283075,0.199158,0.460680,0.015378,1.000000


### Correlation matrix

In [85]:
import plotly.express as px
import plotly.graph_objs as go

fig = go.Figure(data=go.Heatmap(
                   z=corr_matrix.values,
                   x=corr_matrix.columns,
                   y=corr_matrix.columns,
                   colorscale='Viridis',
                   zmin=-1, zmax=1,
                   hoverongaps=False)
              )

for i in range(len(corr_matrix.columns)):
    for j in range(len(corr_matrix.columns)):
        fig.add_annotation(x=corr_matrix.columns[i], y=corr_matrix.columns[j], 
                           text="{:.2f}".format(corr_matrix.iloc[j, i]), 
                           showarrow=False, font=dict(color='white'))

fig.update_layout(
    title='Correlation Matrix',
    width=800,
    height=800,
    xaxis_title='Variables',
    yaxis_title='Variables')

fig.show()


### Scatterplot

#### (example for migration and CPI)

In [61]:
# def ols_func(y_1, x_1):
#     fig = px.scatter(merged, x = x_1, y = 'y_1', trendline='ols', hover_name='Region', color='Region')

#     fig.update_layout(#title='y vs. x',
#                   xaxis_title='x',
#                   yaxis_title='y')

#     fig.show()

In [62]:
# ols_func('migration', 'population_growth')

In [76]:
fig = px.scatter(merged, x=['cpi'], y='migration', trendline='ols', hover_name='Region', color='Region')

fig.update_layout(title='Migration vs. CPI',
                  xaxis_title='gni_and_gdp',
                  yaxis_title='CPI')

fig.show()

The graph shows a scatter plot of the relationship between the Consumer Price Index (CPI) and migration in various regions. Each point on the plot represents a region and its corresponding CPI and migration values. The trendline drawn using the Ordinary Least Squares (OLS) method represents the overall trend in the data. The graph allows us to visually assess if there is any relationship between the two variables and if so, what type of relationship it is (positive, negative or no relationship).

---

## 3. Regression analysis

In [70]:
# Define the dependent variable (migration) and independent variable (CPI)
y = merged['migration']
X = merged[['population_growth','cpi','gni_and_gdp','hdi','labour_force', 'wages']]

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              migration   R-squared:                       0.443
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                     21.32
Date:                Fri, 21 Apr 2023   Prob (F-statistic):           2.47e-18
Time:                        11:45:07   Log-Likelihood:                -1732.0
No. Observations:                 168   AIC:                             3478.
Df Residuals:                     161   BIC:                             3500.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              2.004e+06   2.37e+0

This is an OLS (Ordinary Least Squares) regression analysis with migration as the dependent variable and population_growth, cpi, gni_and_gdp, hdi, and labour_force as independent variables.

- The R-squared value of 0.307 suggests that the model explains 30.7% of the variation in the migration variable. This value indicates that the model has a moderate fit to the data.


- The coefficients of the independent variables show the direction and magnitude of the relationships between them and the dependent variable. The constant term is 1.513e+06, which is the estimated value of the dependent variable when all the independent variables are equal to zero.


- The coefficient for population_growth is 0.0777, but its p-value (0.314) suggests that this variable is not statistically significant in the model.


- The coefficient for cpi is -1.564e+04, which suggests that as the cpi variable decreases, the migration variable increases. The p-value for cpi is 0.000, indicating that the variable is statistically significant in the model.


- The coefficient for gni_and_gdp is -0.0001, but its p-value (0.788) suggests that this variable is not statistically significant in the model.


- The coefficient for hdi is 1.183e+05, which suggests that as the hdi variable increases, the migration variable increases. The p-value for hdi is 0.000, indicating that the variable is statistically significant in the model.


- The coefficient for labour_force is -509.0275, which suggests that as the labour_force variable decreases, the migration variable increases. The p-value for labour_force is 0.012, indicating that the variable is statistically significant in the model.

- The coefficient for wages is 0.1162. For a one unit increase in wages, migration levels increase by 0.1162. This coefficient is statistically significant (p-value < 0.001) and positive, indicating that higher wages are associated with higher migration levels.

- The F-statistic of 14.32 with a corresponding p-value of 1.32e-11 indicates that at least one independent variable in the model is significant.


- The Omnibus test and Jarque-Bera test suggest that the errors in the model are not normally distributed, and the Durbin-Watson test shows that there is a possibility of positive autocorrelation in the model's residuals.


Overall, the regression analysis suggests that cpi, hdi, and labour_force have a significant relationship with migration. However, the relatively low R-squared value and the issues with the model's residuals indicate that there may be other factors affecting the migration variable that are not accounted for in this model.

### F-test

In [71]:
y = merged['migration']
X = merged[['population_growth','cpi','gni_and_gdp','hdi','labour_force', 'wages']]

f_scores, p_values = f_regression(X, y)

display('F-scores:', f_scores)
display('p-values:', p_values)

'F-scores:'

array([ 6.10704891, 42.62631683,  1.91234205,  2.43068787, 18.05244534,
       15.81675817])

'p-values:'

array([1.44738431e-02, 7.72443770e-10, 1.68560677e-01, 1.20885129e-01,
       3.57326543e-05, 1.03991066e-04])

Looking at the output, we can see that `population_growth`, `cpi`, `hdi`, `labour_force`, and `wages` have p-values less than 0.05, indicating that they have statistically significant relationships with the dependent variable, `migration`. The F-scores also suggest that these variables are relatively important in explaining the variation in `migration`, with `cpi` having the highest F-score.

On the other hand, `gni_and_gdp` has a p-value greater than 0.05, indicating that there is no statistically significant relationship between this variable and `migration`. The low F-score for this variable also suggests that it may not be very important in explaining the variation in `migration`.

### Breusch-Pagan

We were concerned about heteroskedasticity in our regression analysis and the White test is not applicable due to the issue with degrees of freedom, one alternative test we could consider is the Breusch-Pagan test. The Breusch-Pagan test is similar to the White test in that it tests for heteroskedasticity in the residuals of a regression model, but it is more robust to small sample sizes.

In [72]:
y = merged['migration']
X = merged[['population_growth','cpi','gni_and_gdp','hdi','labour_force', 'wages']]

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

bp_test = het_breuschpagan(model.resid, X)
print("Breusch-Pagan test statistic:", bp_test[0])
print("Breusch-Pagan p-value:", bp_test[1])

Breusch-Pagan test statistic: 38.88358356845964
Breusch-Pagan p-value: 7.543677373024799e-07


The Breusch-Pagan test is a statistical test used to detect heteroscedasticity in a regression model. Heteroscedasticity occurs when the variance of the errors in a regression model is not constant across all observations, which violates one of the assumptions of a linear regression model.

The test statistic in this case is 28.687 and the p-value is 2.67e-05. The p-value is less than the conventional significance level of 0.05, indicating that there is strong evidence of heteroscedasticity in the regression model. Therefore, the assumption of constant variance of errors across observations is violated and the results of the regression model should be interpreted with caution.

---

### Other tests:

There are several statistical tests that can be useful for analyzing the relationship between the variables in your project. Here are a few examples:

1. Pearson correlation coefficient: This test measures the linear relationship between two continuous variables, such as the correlation between population growth and migration.


2. Spearman correlation coefficient: This test measures the monotonic relationship between two continuous or ordinal variables. It can be useful when the relationship between the variables is non-linear or the data is skewed.


3. ANOVA (Analysis of Variance): This test is used to compare the means of two or more groups. It can be useful for analyzing differences in economic variables across regions.

#### 1. Pearson correlation coefficient

In [73]:
variables = ['population_growth', 'cpi', 'gni_and_gdp', 'hdi', 'labour_force', 'wages']

for var in variables:
    x = merged[var]
    y = merged['migration']
    corr_coef, p_value = pearsonr(x, y)
    print(f"{var} - Pearson correlation coefficient: {corr_coef:.5f}, p-value: {p_value:.5f}")


population_growth - Pearson correlation coefficient: 0.18837, p-value: 0.01447
cpi - Pearson correlation coefficient: -0.45202, p-value: 0.00000
gni_and_gdp - Pearson correlation coefficient: -0.10672, p-value: 0.16856
hdi - Pearson correlation coefficient: 0.12013, p-value: 0.12089
labour_force - Pearson correlation coefficient: -0.31318, p-value: 0.00004
wages - Pearson correlation coefficient: 0.29495, p-value: 0.00010


Based on the Pearson correlation coefficient and p-values, we can make the following interpretations:

- Population_growth has a positive correlation with migration (coefficient=0.18837, p-value=0.01447). This means that regions with a higher population growth tend to have higher levels of migration.


- CPI has a negative correlation with migration (coefficient=-0.45202, p-value=0.00000). This means that regions with higher levels of consumer price inflation tend to have lower levels of migration.


- GNI_and_GDP has a weak negative correlation with migration (coefficient=-0.10672, p-value=0.16856). However, since the p-value is greater than 0.05, we cannot conclude that there is a statistically significant relationship between these two variables.


- HDI has a weak positive correlation with migration (coefficient=0.12013, p-value=0.12089). However, since the p-value is greater than 0.05, we cannot conclude that there is a statistically significant relationship between these two variables.


- Labour_force has a negative correlation with migration (coefficient=-0.31318, p-value=0.00004). This means that regions with higher levels of labour force tend to have lower levels of migration.


- Wages has a moderate positive correlation between wages and migration levels (r = 0.295), and the p-value is highly statistically significant (p = 0.0001), indicating that this correlation is very unlikely to have occurred by chance.

#### 2. Spearman correlation coefficient

In [68]:
for column in merged.columns:
    coef, pval = spearmanr(merged[column], merged['migration'])
    print(f"{column} - Spearman correlation coefficient: {coef:.5f}, p-value: {pval:.5f}")


year - Spearman correlation coefficient: -0.24970, p-value: 0.00110
Region - Spearman correlation coefficient: -0.03069, p-value: 0.69290
migration - Spearman correlation coefficient: 1.00000, p-value: 0.00000
population_growth - Spearman correlation coefficient: 0.23809, p-value: 0.00189
cpi - Spearman correlation coefficient: -0.45012, p-value: 0.00000
gni_and_gdp - Spearman correlation coefficient: 0.08442, p-value: 0.27661
hdi - Spearman correlation coefficient: 0.00748, p-value: 0.92332
labour_force - Spearman correlation coefficient: -0.29855, p-value: 0.00008
wages - Spearman correlation coefficient: 0.22308, p-value: 0.00365


The Spearman correlation coefficient is a nonparametric test that assesses the strength and direction of association between two ranked variables. In this case, the results show:

- year: The Spearman correlation coefficient of -0.24970 and a p-value of 0.00110 suggests that there is a weak negative correlation between the year and migration. This means that migration tends to decrease slightly over the years.


- Region: The Spearman correlation coefficient of -0.03069 and a p-value of 0.69290 suggests that there is a very weak negative correlation between Region and migration. However, this p-value is not significant, indicating that there is no evidence of a correlation between these variables.


- migration: The Spearman correlation coefficient of 1.00000 and a p-value of 0.00000 indicates a perfect positive correlation between migration and itself, which is expected.


- population_growth: The Spearman correlation coefficient of 0.23809 and a p-value of 0.00189 suggests that there is a weak positive correlation between population growth and migration. This means that countries with higher population growth tend to have slightly higher levels of migration.


- cpi: The Spearman correlation coefficient of -0.45012 and a p-value of 0.00000 suggests that there is a moderate negative correlation between cpi and migration. This means that countries with higher levels of corruption tend to have lower levels of migration.


- gni_and_gdp: The Spearman correlation coefficient of 0.08442 and a p-value of 0.27661 suggests that there is a very weak positive correlation between gni_and_gdp and migration. However, this p-value is not significant, indicating that there is no evidence of a correlation between these variables.


- hdi: The Spearman correlation coefficient of 0.00748 and a p-value of 0.92332 suggests that there is no evidence of a correlation between hdi and migration.


- labour_force: The Spearman correlation coefficient of -0.29855 and a p-value of 0.00008 suggests that there is a moderate negative correlation between labour force and migration. This means that countries with higher levels of labour force tend to have lower levels of migration.


- wages: The Spearman correlation coefficient of 0.22308 and a p-value of 0.00365 suggests that there is a weak positive correlation between wages and migration. This means that countries with higher wages tend to have slightly higher levels of migration.

#### 3. ANOVA

- The null hypothesis for the one-way ANOVA test is that there is no significant difference in the means of the variable across the groups.
- The alternative hypothesis is that there is at least one group with a significantly different mean.

In [74]:
grouped = merged.groupby('Region')

variables = ['cpi', 'gni_and_gdp', 'hdi', 'labour_force', 'wages']
for variable in variables:
    data = [group[1][variable].values for group in grouped]
    f_value, p_value = f_oneway(*data)
    print(f"{variable} ANOVA Results:\nF-Value: {f_value:.4f}\nP-Value: {p_value:.5f}\n")

cpi ANOVA Results:
F-Value: 15.1425
P-Value: 0.00000

gni_and_gdp ANOVA Results:
F-Value: 76338.6511
P-Value: 0.00000

hdi ANOVA Results:
F-Value: 13.5216
P-Value: 0.00000

labour_force ANOVA Results:
F-Value: 27.1706
P-Value: 0.00000

wages ANOVA Results:
F-Value: 5.7288
P-Value: 0.00000



The ANOVA results indicate that there are significant differences in the mean values of the economic variables (cpi, gni_and_gdp, hdi, and labour_force) across regions. Specifically, for each variable, the F-value is significantly different from 1 (indicating the existence of significant variation in the means of the groups), and the p-value is less than 0.05 (indicating that the observed differences are unlikely to be due to chance). Therefore, we can conclude that there are significant differences in these economic variables across the different regions.